In this script we will extract metrics for each commit. For that you need to download and install Understand Tool, available here https://scitools.com/trial-download-3/ . To run you will need license to Understand, student get free in https://scitools.com/non-commercial-license/

In [1]:
pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install JPype1

Note: you may need to restart the kernel to use updated packages.


In [3]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [4]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil

In [5]:
def getCommitsFrom(project):
    projectPath = "repositories/"+project
    repository = Repo(projectPath)
    try:
        repository.git.checkout("main", "-f")
    except:
        repository.git.checkout("master", "-f")
    iterCommits = repository.iter_commits()
    commits = []
    for c in iterCommits:
        commits.append(c)
    return commits

In [6]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [7]:
def extractMetricsWithUnderstand(owner, project):
    understandPath = "understand\\SciTools\\bin\\pc-win64\\und"
    os.system('cmd /c "{0} create -languages java {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} add {1} {2}"'.format(understandPath, "repositories/"+owner+"/"+project, project))
    os.system('cmd /c "{0} settings -metrics all {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} settings -metricsOutputFile {1}.csv {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} -quiet analyze {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} metrics {1}"'.format(understandPath, project))

In [8]:
def method_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Method")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [9]:
def class_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Class")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [10]:
def project_level_metrics(dataframe, owner, project):
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Package"]
    packages = len(dataframe)
    dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["numberJavaFiles"] = numberJavaFiles
    dataframe["packages"] = packages
    return dataframe

In [11]:
def readMetricsFromCsv(project):
    dtype = {'Kind': str, 'Name': str, 'AvgCyclomatic': np.float64, 'AvgCyclomaticModified': np.float64,'AvgCyclomaticStrict': np.float64, 'AvgEssential': np.float64, 'AvgLine': np.float64, 'AvgLineBlank': np.float64,'AvgLineCode': np.float64, 'AvgLineComment': np.float64, 'CountClassBase': np.float64, 'CountClassCoupled': np.float64,'CountClassCoupledModified': np.float64, 'CountClassDerived': np.float64, 'CountDeclClass': np.float64,'CountDeclClassMethod': np.float64, 'CountDeclClassVariable': np.float64,'CountDeclExecutableUnit': np.float64, 'CountDeclFile': np.float64, 'CountDeclFunction': np.float64,'CountDeclInstanceMethod': np.float64, 'CountDeclInstanceVariable': np.float64,'CountDeclMethod': np.float64, 'CountDeclMethodAll': np.float64, 'CountDeclMethodDefault': np.float64,'CountDeclMethodPrivate': np.float64, 'CountDeclMethodProtected': np.float64,'CountDeclMethodPublic': np.float64, 'CountInput': np.float64,'CountLine': np.float64, 'CountLineBlank': np.float64,'CountLineCode': np.float64, 'CountLineCodeDecl': np.float64, 'CountLineCodeExe': np.float64,'CountLineComment': np.float64, 'CountOutput': np.float64, 'CountPath': np.float64, 'CountPathLog': np.float64,'CountSemicolon': np.float64, 'CountStmt': np.float64, 'CountStmtDecl': np.float64, 'CountStmtExe': np.float64,'Cyclomatic': np.float64, 'CyclomaticModified': np.float64, 'CyclomaticStrict': np.float64, 'Essential': np.float64, 'Knots': np.float64, 'MaxCyclomatic': np.float64, 'MaxCyclomaticModified': np.float64,'MaxCyclomaticStrict': np.float64, 'MaxEssential': np.float64,'MaxEssentialKnots': np.float64,'MaxInheritanceTree': np.float64, 'MaxNesting': np.float64, 'MinEssentialKnots': np.float64,'PercentLackOfCohesion': np.float64, 'PercentLackOfCohesionModified': np.float64,'RatioCommentToCode': np.float64, 'SumCyclomatic': np.float64, 'SumCyclomaticModified': np.float64,'SumCyclomaticStrict': np.float64, 'SumEssential': np.float64}
    dataframe = pd.read_csv(project+".csv", dtype=dtype)
    return dataframe

In [12]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    
    dataframe = readMetricsFromCsv(project)
    
    method_level = method_level_metrics(dataframe.copy(), owner, project)
    method_level["level"] = "method"
    method_level.set_index("level")

    
    class_level = class_level_metrics(dataframe.copy(), owner, project)
    class_level["level"] = "class"
    class_level.set_index("level")    
    
    project_level = project_level_metrics(dataframe.copy(), owner, project)
    project_level["level"] = "project"
    project_level.set_index("level")
    
    return pd.concat([method_level, class_level, project_level], axis=0)

In [13]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [18]:
def getMetrics(commit, owner, project):
    metrics = getUnderstandMetrics(owner, project)
    metrics["commitSha"] = commit.hexsha
    metrics["commitDate"] = commit.authored_datetime
    return metrics

In [19]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".csv")
    shutil.rmtree(sample + ".und")

In [20]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [21]:
def replaceSamplePathForWindowsLike(sample):
    return sample.replace("/", "\\")

In [22]:
def extractMetricsByCommit(sample, commit):
    owner, project = sample.split("\\")
    checkoutTo(sample, commit.hexsha)
    print("commit ======= " + commit.hexsha)
    metrics = getMetrics(commit, owner, project)
    return metrics

In [23]:
def extractMetricsForAllCommits(commits, sample):
    allCommits = pd.DataFrame()
    for index, commit in enumerate(commits):
        metrics = extractMetricsByCommit(sample, commit)
        allCommits = allCommits.append(metrics, ignore_index=True)
        print("{0}% of commits completed from sample {1}".format((index/len(commits) * 100), sample))
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)
    return allCommits

In [25]:
exclude = []
def metricsByCommit(samples):
    for index, sample in enumerate(samples):
        if (sample in exclude):
            continue
        print(sample)
        sample = replaceSamplePathForWindowsLike(sample)
        
        printStatus(index+1, len(samples))
        
        createDirectoryIfNotExists("metrics")
        
        commits = getCommitsFrom(sample)
        commits.reverse()
        
        owner, project = sample.split("\\")
        
        createDirectoryIfNotExists("metrics\\"+owner)

#         ######### if the script crash, copy the commit sha of the last sucess analyses and paste below. And then uncomment this code
#         for index, commit in enumerate(commits):
#             if commit.hexsha == "2088b83e376319902f7e36a09df745fb49dab8c2":
#                 position = index
#         commits = commits[position+1:]
        
        allCommits = extractMetricsForAllCommits(commits, sample)
        
        deleteUnusedFiles(project)
        
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)

In [26]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [27]:
samples["full_name"] = samples["framework"] + "/" +samples["path"]

In [28]:
metricsByCommit(samples["full_name"].unique())

googlesamples/io2014-codelabs
0.53475935828877% Completed samples
commit ======= 93d363b4f72eb520dd80520ed6ce331ead154a4d


<ipython-input-8-037bca295bde>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["sample"] = owner+"/"+project
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-10-78c09f80697d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

0.0% of commits completed from sample googlesamples\io2014-codelabs
commit ======= b1039047c1c5a72efe69af133f86684c5607c32d
6.666666666666667% of commits completed from sample googlesamples\io2014-codelabs
commit ======= 09a8403669d6845d5a5b7b9fa539a21633112b2f
13.333333333333334% of commits completed from sample googlesamples\io2014-codelabs
commit ======= 37978bf45b229d1ec54a24d99f127cd72d6b8698
20.0% of commits completed from sample googlesamples\io2014-codelabs
commit ======= 1a2432d42f698ccd6c6db0ad866960b8a63bb028
26.666666666666668% of commits completed from sample googlesamples\io2014-codelabs
commit ======= d5af0024eec2237a41565fc030fdc2627c0f6ba8
33.33333333333333% of commits completed from sample googlesamples\io2014-codelabs
commit ======= 0c8eb86958cf435f667d2b36e7c1af6f66d20818
40.0% of commits completed from sample googlesamples\io2014-codelabs
commit ======= b947ada7b763ddcc8b31838c991fe42bafa3fdff
46.666666666666664% of commits completed from sample googlesamples\io201

12.5% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 56ffc7062e515be066f97f1866056c87f1071873
12.962962962962962% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= b9ab30adf5f2df388965cc9e540af26e9d4069b3
13.425925925925927% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 51db692d23e0be59e323f8eeb4a9a0eeb22a2e66
13.88888888888889% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 73808ca49f5b8b48df9ce86792c147f87354e14d
14.351851851851851% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 2ade8403dec020718fd433297796f5b86f9bb445
14.814814814814813% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= e05371f0e0f82e466aeb0308bf805869d13f30a1
15.277777777777779% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= fdf6746be16e82869008c51c496

38.88888888888889% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= b43c8517ca7c721579cc11ec4a50c9e85a40b89c
39.351851851851855% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 5e8cabffc0d59a59f8f099ed8889470c3db69272
39.81481481481482% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 43034c8189700b2cbddf41d2a5762c9fec5b266f
40.27777777777778% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= b6283fb2b2a7442dddd7a7417835c7bfcee328a8
40.74074074074074% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= c00a14f67bdbffd501dfe7080c8d0bd106ae4cb0
41.2037037037037% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 0d3b158fd0968079b566632210ac349408240286
41.66666666666667% of commits completed from sample googlesamples\androidtv-sample-inputs
commit ======= 34c400a3768ee2f7723

...

75.8800521512386% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= dd7766650ec4f8bc1b10faea1c17843ca4533c64
76.01043024771839% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 455d02fec5be8ac6ebcca03d09a9ff7a2691c78c
76.14080834419818% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= afc3f6cd6596fc215da7fbf66020ac7285d44e3a
76.27118644067797% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 86a3f1fe857cac5486fef5332efe16a13cfeda53
76.40156453715777% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= edbd3104e3cabeb699ca73e23379a621c73493f0
76.53194263363756% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 78eb9ac6ce3198c1580d690f6e80b6c1b6206311
76.66232073011734% of commits completed from sa

82.65971316818775% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 4c3a440b4bcc44184e5752e0c44053a4ae35089a
82.79009126466754% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= fa9e3455d779b48b279c36e01d21891da0b33eeb
82.92046936114733% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 94cae9a0db415b81040d28db1460f4e65e932a4a
83.05084745762711% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= cc21626868d8e53c713710e2cce3bf18fd16d879
83.1812255541069% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 683dca0f309b76233f727d9d816a80fdcc4ad2f3
83.31160365058669% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= f1e8115d7c65e692a869a23328c5829db90ebbd0
83.44198174706649% of commits completed from sa

commit ======= 9705ce335e99bc5f15aa4b655703efd3e623058a
89.4393741851369% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 3c2c677153f2890a7c461a3519fa3b6d3d00ae07
89.56975228161669% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 69e698a3554fcb33c8e563d082d1410b1929d664
89.70013037809647% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 64073e12bfd4d70962ec81e43ae19554a2bc83a4
89.83050847457628% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= da3b605f702d5112dc89d7679c2150e3996fb00e
89.96088657105607% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 4594af425c219a83a189532c69a978748f0e75c8
90.09126466753585% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= e945058553bea0d2987af372fbdfaedb

96.21903520208605% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 2b115738e19fac9bb7708c282871904d7cc0a30b
96.34941329856585% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 4c7ea138f3bd6a10cc7a70c066cdba67a2876a27
96.47979139504564% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 82b28817a2740b3c65c23fdd51292a2a7364eb24
96.61016949152543% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= aae66904e0f955d0d99e71cbf6faf82b1bdf09bd
96.74054758800521% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 3a3b9b7f232da056ff322dd78ff883f00a050a53
96.87092568448502% of commits completed from sample spring-cloud-samples\spring-cloud-contract-samples
commit ======= 3a443f4b58f1c3390a8dacadbf20e1a75983039f
97.0013037809648% of commits completed from sa

33.33333333333333% of commits completed from sample spring-cloud-samples\sample-zuul-filters
commit ======= 541d265116bab308c235cd7499324c3ec4f1f71b
44.44444444444444% of commits completed from sample spring-cloud-samples\sample-zuul-filters
commit ======= 32314b1411ea63bbfc30e2c92a73de0c84b6be64
55.55555555555556% of commits completed from sample spring-cloud-samples\sample-zuul-filters
commit ======= 1d1b3c392f43c829e41e515d183674c2ecaf4902
66.66666666666666% of commits completed from sample spring-cloud-samples\sample-zuul-filters
commit ======= 700c299dedec417cefa16f4f7cfe48c78eab5fe3
77.77777777777779% of commits completed from sample spring-cloud-samples\sample-zuul-filters
commit ======= 98171a1057ffa1620294f5acbdf064aa7e554050
88.88888888888889% of commits completed from sample spring-cloud-samples\sample-zuul-filters
spring-cloud-samples/spring-cloud-config-vault
98.93048128342245% Completed samples
commit ======= 6786b9175dc0f13289990b9da21fbb26bffa480a
0.0% of commits comple